In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
import numpy as np

In [2]:
from catboost import CatBoostClassifier

In [3]:
train_df_original = pd.read_csv('TRAIN_FINAL.csv')
test_df = pd.read_csv('Test_v1.csv')

In [14]:
positive_dataset = train_df_original[train_df_original['TARGET']==0]

negative_dataset = train_df_original[train_df_original['TARGET']==1]

sample_positive5 = positive_dataset.sample(n=1763*5,random_state=42)
train_df5 = pd.concat([sample_positive,negative_dataset])
train_df5 = train_df.sample(frac = 1,random_state=42)

sample_positive10 = positive_dataset.sample(n=1763*10,random_state=43)
train_df10 = pd.concat([sample_positive,negative_dataset])
train_df10 = train_df.sample(frac = 1,random_state=42)

sample_positive15 = positive_dataset.sample(n=1763*15,random_state=44)
train_df15 = pd.concat([sample_positive,negative_dataset])
train_df15 = train_df.sample(frac = 1,random_state=42)

In [15]:
X_5 = train_df5.drop(['TARGET'],axis=1)

y_5 = train_df5['TARGET']

X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split( X_5, y_5, test_size=0.33, random_state=42,stratify = y_5)


X_10 = train_df10.drop(['TARGET'],axis=1)

y_10 = train_df10['TARGET']

X_train_10, X_test_10, y_train_10, y_test_10 = train_test_split( X_10, y_10, test_size=0.33, random_state=43,stratify = y_10)

X_15 = train_df15.drop(['TARGET'],axis=1)

y_15 = train_df15['TARGET']

X_train_15, X_test_15, y_train_15, y_test_15 = train_test_split( X_15, y_15, test_size=0.33, random_state=44,stratify = y_15)

In [72]:
#5

cat_5 = CatBoostClassifier(class_weights=[1,5],n_estimators=500,depth=6, 
    l2_leaf_reg = 14)

cat_5.fit(X_train_5, y_train_5,eval_set=(X_test_5, y_test_5),
                               use_best_model=True,verbose_eval=False)

pred = cat_5.predict(X_test_5)
print(roc_auc_score(y_test_5,pred))


#10

cat_10 = CatBoostClassifier(class_weights=[1,10],n_estimators=500,depth=6, 
    l2_leaf_reg = 14)

cat_10.fit(X_train_10, y_train_10,eval_set=(X_test_10, y_test_10),
                               use_best_model=True,verbose_eval=False)

pred = cat_10.predict(X_test_10)
print(roc_auc_score(y_test_10,pred))


#15

cat_15 = CatBoostClassifier(class_weights=[1,15],n_estimators=500,depth=6, 
    l2_leaf_reg = 14)

cat_15.fit(X_train_15, y_train_15,eval_set=(X_test_15, y_test_15),
                               use_best_model=True,verbose_eval=False)

pred = cat_15.predict(X_test_15)
print(roc_auc_score(y_test_15,pred))


0.731508684389
0.754341308346
0.756819693356


In [44]:
#tune threshold

In [73]:
#stacking test

# stacking dataset

cat_5_out =  cat_5.predict_proba(X_test_15)[:,1]
cat_10_out = cat_10.predict_proba(X_test_15)[:,1]
cat_15_out = cat_15.predict_proba(X_test_15)[:,1]
pred = ((cat_5_out + cat_10_out + cat_15_out)/3 > 0.44).astype('int')
print(roc_auc_score(y_test_15,pred))

0.7759031398


In [74]:
for i in np.arange(0.4,0.5,0.005):
    pred = ((cat_5_out + cat_10_out + cat_15_out)/3 > i).astype('int')
    print(i,roc_auc_score(y_test_15,pred))

0.4 0.774779715517
0.405 0.774522780942
0.41 0.776327820167
0.415 0.776929992121
0.42 0.776673057545
0.425 0.777962455657
0.43 0.773495928621
0.435 0.775300967846
0.44 0.7759031398
0.445 0.777450358468
0.45 0.778395995837
0.455 0.777279954732
0.46 0.777366485572
0.465 0.777538956598
0.47 0.776680440723
0.475 0.776422915493
0.48 0.777626373419
0.485 0.777455083702
0.49 0.776682508012
0.495 0.774793005237


In [65]:
ind = test_df['index']
test = test_df.drop(['index'],axis=1)

In [62]:
cat_5_out =  cat_5.predict_proba(test)[:,1]
cat_10_out = cat_10.predict_proba(test)[:,1]
cat_15_out = cat_15.predict_proba(test)[:,1]

y_out = ((cat_5_out + cat_10_out + cat_15_out)/3 > 0.4).astype('int')
dfdict={}

dfdict["index"]=ind
dfdict["TARGET"]=y_out

df=pd.DataFrame(dfdict)
df.to_csv("solution.csv",index=False,columns=["index","TARGET"])

In [ ]:
'''0.766 5 times oversampled
0.769291061394 10 times oversampled
0.773670762263 class_weights=[1,10],n_estimators=500,depth=6, 
    l2_leaf_reg = 14
0.77049245203  class_weights=[1,10],n_estimators=500,depth=8, 
    l2_leaf_reg = 14
 0.761128515721   (class_weights=[1,20],n_estimators=1000,depth=2, 
    l2_leaf_reg = 14
0.761386040951 class_weights=[1,20],n_estimators=500,depth=6, 
    l2_leaf_reg = 14
0.776078268769 class_weights=[1,5],n_estimators=500,depth=6, 
    l2_leaf_reg = 14  0.75977 on lb

'''